In [7]:
import copy
import datetime as dt
from pathlib import Path
import requests
import warnings

import holidays
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import metrics
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


pd.options.mode.chained_assignment = None  # default='warn'
matplotlib.rcParams.update({'font.size': 15})
warnings.simplefilter("ignore")

In [3]:
BASE_PATH = Path.cwd().parent
DATA_PATH = BASE_PATH / "data"

train = pd.read_csv(DATA_PATH / "train.csv")
test = pd.read_csv(DATA_PATH / "test.csv")

In [4]:
df = pd.concat([train, test], axis=0)
df['log'] = np.log(df['num_sold'])

# add date
df['date']  = pd.to_datetime(df['date'])
df['year']  = df['date'].dt.year
df['week']  = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['day']   = df['date'].dt.day
df['time_no'] = (df['date'] - dt.datetime(2017, 1, 1)) // dt.timedelta(days=1)
df.loc[df['date'] > dt.datetime(2020, 2, 29), 'time_no'] -= 1

In [5]:
# Define the years and countries
years = [2017,2018,2019,2020,2021,2022]
countries = train.country.unique()

dfs = []
# Generate holidays for each country and year
for year in years:
    for country in countries:
        for date, holiday_name in sorted(holidays.CountryHoliday(country, years=year).items()):
            
            df_0 = pd.DataFrame({"date": [date], "country": [
                              country]})
            dfs.append(df_0)

# Concatenate all the DataFrames
df_holidays = pd.concat(dfs, ignore_index=True)

# Convert 'date' column to datetime
df_holidays['date'] = pd.to_datetime(df_holidays['date'])
df_holidays['tmp'] = 1

#remove certain holidays since the data doesn't have "holiday upturn" on these holidays
df_holidays = df_holidays[~((df_holidays['date'].dt.month.isin([2,4,5,8,10])) & (df_holidays['country'] == 'Canada'))]
# remove New Year and Christmas Holiday because I will handle them seperately
for country in ['Argentina', 'Canada', 'Estonia', 'Spain']:
    for year in years:
        df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime(
            f'{year}-01-01')) & (df_holidays['country'] == country))]
df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime(
    '2017-01-02')) & (df_holidays['country'] == 'Spain'))]

for country in ['Argentina', 'Canada', 'Estonia', 'Spain']:
    for year in years:
        df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime(
            f'{year}-12-25')) & (df_holidays['country'] == country))]
df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime(
    '2022-12-26')) & (df_holidays['country'] == 'Spain'))]

# Canada and Estonia has additional holiday on 26th following Christmas. I will handle them separately
for country in ['Canada', 'Estonia']:
    for year in years:
        df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime(
            f'{year}-12-26')) & (df_holidays['country'] == country))]

#Canada has additional holiday on 27, 28. I remove them and it increases cross validation
df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime('2020-12-28')) & (df_holidays['country'] == 'Canada'))]
df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime('2021-12-27')) & (df_holidays['country'] == 'Canada'))]
df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime('2021-12-28')) & (df_holidays['country'] == 'Canada'))]
df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime('2022-12-27')) & (df_holidays['country'] == 'Canada'))]

#it seems that Japan doesn't celebrate on this day. I remove it and it increases cross validation
df_holidays = df_holidays[~((df_holidays['date'] == pd.to_datetime(
            f'2018-12-24')) & (df_holidays['country'] == 'Japan'))]


In [8]:
def get_gdp_per_capita(country, year):
    alpha3 = {'Argentina': 'ARG', 'Canada': 'CAN',
              'Estonia': 'EST', 'Japan': 'JPN', 'Spain': 'ESP'}
    url = "https://api.worldbank.org/v2/country/{0}/indicator/NY.GDP.PCAP.CD?date={1}&format=json".format(
        alpha3[country], year)
    response = requests.get(url).json()
    return response[1][0]['value']


gdp = []
for country in train.country.unique():
    row = []
    for year in range(2017, 2023):
        row.append(get_gdp_per_capita(country, year))
    gdp.append(row)

gdp = np.array(gdp)
gdp /= np.sum(gdp)

rel_gdp_df = pd.DataFrame(gdp, index=train.country.unique(), columns=range(2017, 2023))
df['rel_gdp'] = df.apply(lambda s: rel_gdp_df.loc[s.country, s.date.year], axis=1)
df['log_rel_gdp'] = np.log(df['rel_gdp'])

In [9]:
# holiday
holiday_diff = [np.exp(-(i - 4.5) ** 2 / 8.5) for i in range(11)]
holidays_columns = ['holiday']
df['holiday'] = 0
for day_no, diff in enumerate(holiday_diff):
    shifted = df_holidays.copy()
    shifted['date'] = shifted['date'] + dt.timedelta(days=day_no)
    df = pd.merge(df, shifted, on=['country', 'date'], how='left')
    df['tmp'].fillna(0, inplace=True)
    df['holiday'] += df['tmp'] * diff
    df.drop(columns='tmp', inplace=True)
# New Year's holiday
special_date_columns = []
for day in range(25, 32):
    column = 'day_12_{}'.format(day)
    df[column] = ((df['month'] == 12) & (df['day'] == day) &
                  (df['country'] != 'Japan')).astype(float)
    special_date_columns.append(column)
for day in range(1, 11):
    column = 'day_1_{}'.format(day)
    df[column] = ((df['month'] == 1) & (df['day'] == day) & 
                  ((df['year'] == 2017) | (df['country'] != 'Japan'))).astype(float)
    special_date_columns.append(column)
holiday_countries = ['Estonia','Canada']
column = 'holiday_1226'
holiday_diff = [np.exp(-(i - 4.5) ** 2 / 8.5) for i in range(11)]
df[column] = 0
for day_no, diff in enumerate(holiday_diff):
    df.loc[(df['country'].isin(holiday_countries)) & (df['month'] == 12) & (df['day'] == 26 + day_no), column] = diff
    df.loc[(df['country'].isin(holiday_countries)) & (df['month'] ==  1) & (df['day'] == -5 + day_no), column] = diff

special_date_columns.append(column)

In [10]:
# sin wave
year_columns = ['year_sin_1', 'year_cos_1', 'year_sin_0.5', 'year_cos_0.5']
df['year_sin_1']   = np.sin(np.pi * df['time_no'] / 182.5)
df['year_cos_1']   = np.cos(np.pi * df['time_no'] / 182.5)
df['year_sin_0.5'] = np.sin(np.pi * df['time_no'] / 365.0)
df['year_cos_0.5'] = np.cos(np.pi * df['time_no'] / 365.0)

# prodcut feature.
#I did crossvalidation and find that we only need simple sin and cos wave
product_year_columns = []
for product in train['product'].unique():
    df[product] = (df['product'] == product).astype(float)
    product_sin = '{}_sin'.format(product)
    product_cos = '{}_cos'.format(product)
    if product == 'Using LLMs to Train More LLMs' or product == 'Using LLMs to Win Friends and Influence People':
        df[product_sin] = df[product] * df['year_sin_0.5']
        df[product_cos] = df[product] * df['year_cos_0.5']

        product_year_columns.append(product_sin)
        product_year_columns.append(product_cos)
    elif product == 'Using LLMs to Write Better' or product== 'Using LLMs to Improve Your Coding':
        df[product_sin] = df[product] * df['year_sin_1']
        product_year_columns.append(product_sin)    
    else:
        df[product_cos] = df[product] * df['year_cos_1']
        product_year_columns.append(product_cos)

In [11]:
# make month flag in 2020
featured_month_columns = []
for month in range(3, 11):
    column = 'month_2020_{}'.format(month)
    df[column] = ((df['year'] == 2020) & (df['month'] == month)).astype(float)
    featured_month_columns.append(column)

In [12]:
# week flag
week_columns = []
for week in range(4, 7):
    column = 'week_{}'.format(week)
    df[column] = (df['week'] == week).astype(float)
    week_columns.append(column)
store_columns = []

for store in train.store.unique()[1:]:
    column = 'store_{}'.format(store)
    df[column] = ((df['store'] == store)).astype(float)
    #df[column] = ((df['store'] == store) & (df['year'] != 2020)).astype(float)
    store_columns.append(column)
# product
product_columns = []
for product in train['product'].unique()[1:]:
    column = 'product_{}'.format(product)
    df[column] = (df['product'] == product).astype(float)
    product_columns.append(column)

In [15]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# decide use columns
use_columns = []
use_columns.extend(special_date_columns)
use_columns.extend(product_year_columns)
use_columns.extend(holidays_columns)
use_columns.extend(week_columns)
use_columns.extend(store_columns)
use_columns.extend(product_columns)
use_columns.extend(featured_month_columns)


# learning
df_used = df.copy()
date = dt.datetime(2021, 12, 31)
df_used = df_used.loc[df_used['date'] <= date]
source = df_used[use_columns]
target = df_used['log']-df_used['log_rel_gdp']

model = Pipeline([
    ('standardize', StandardScaler()),
    ('linear_reg', Ridge(alpha=150, tol=0.00001, max_iter=10000))

])
model.fit(source, target)
linear_reg_model = model.named_steps['linear_reg']
coef = pd.DataFrame(linear_reg_model.coef_, columns=['coef'])
coef['column'] = source.columns
coef.set_index('column', inplace=True)
coef.loc['intercept'] = linear_reg_model.intercept_

In [16]:
# show results
df['predict_log'] = model.predict(df[use_columns])+df['log_rel_gdp']
df['predict_exp'] = np.exp(df['predict_log'])


df['smape_log'] = 2 * (df['log'] - df['predict_log']).abs() / (df['log'] + df['predict_log'])

df['smape_exp'] = 2 * (df['num_sold'] - df['predict_exp']).abs() / (df['num_sold'] + df['predict_exp'])

result = df.loc[df['date'] <= date]
print('e smape  = {}'.format(result['smape_exp'].mean()))

e smape  = 0.044295043764811304


In [17]:
result = df.sort_values('id')
result_2 = result.copy()
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Argentina'), 'predict_exp'] *= 3.372
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Spain'), 'predict_exp'] *= 1.6
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Japan'), 'predict_exp'] *= 1.394
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Estonia'), 'predict_exp'] *= 1.651
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Canada'), 'predict_exp'] *= 0.850
result_2 = result_2.loc[result_2['date'] >= dt.datetime(2022, 1, 1), ['id', 'predict_exp']]


#round
result_2['predict_exp'] = np.round(result_2['predict_exp'],0)
result_2.rename(columns={'predict_exp':'num_sold'}).to_csv('submission.csv', header=True, index=False)

In [18]:
result = df.sort_values('id')
result_2 = result.copy()
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Argentina'), 'predict_exp'] *= 3.372
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Spain'), 'predict_exp'] *= 1.6
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Japan'), 'predict_exp'] *= 1.394
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Estonia'), 'predict_exp'] *= 1.651
result_2.loc[(result_2['year'] == 2022)&(result_2['country'] == 'Canada'), 'predict_exp'] *= 0.850


# First loop over products
for i, product in enumerate(train['product'].unique()):
    print(product)
    fig = plt.figure(figsize=(18, 3))

    # Then loop over countries for each product
    for country in train['country'].unique():
        result_graph = result_2.loc[(result['country'] == country) &
                                  (result['store'] == 'Kaggle Learn') & (result['date'] > dt.datetime(2021, 10, 31)) & (result['date'] < dt.datetime(2022, 5, 31))]
        view = result_graph.loc[result_graph['product'] == product]
        plt.plot(view['date'], view['predict_exp'],label=country)
    plt.legend()
    plt
    plt.show(

SyntaxError: incomplete input (178013030.py, line 23)